In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file
client : OpenAI = OpenAI()

In [2]:
import requests
import json

# Assuming you have an API key from financialmodelingprep.com
FMP_API_KEY = "brbkqWx3oG4v9Xbcu6FQ8QO4lIBGeZBm"

# Define financial statement functions
def get_income_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_growth(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


In [3]:
# Map available functions
available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_cash_flow_statement,
    "get_financial_growth": get_financial_ratios
}

In [8]:
from openai.types.beta import Assistant
# Define the main function


  # Creating an assistant with specific instructions and tools
assistant = client.beta.assistants.create(
      instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
      model="gpt-3.5-turbo-1106",
      tools=[
          
            {"type": "function", 
             "function": {"name": "get_income_statement", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}},
           {"type": "function", 
             "function": {"name": "get_balance_sheet", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}},
             {"type": "function", 
             "function": {"name": "get_cash_flow_statement", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}},
             {"type": "function", 
             "function": {"name": "get_key_metrics", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}},
             {"type": "function", 
             "function": {"name": "get_financial_ratios", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}},
             {"type": "function", 
             "function": {"name": "get_financial_growth", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}, 
             "period": {"type": "string"}, 
             "limit": {"type": "integer"}}}}}
             
          ]
  )

In [9]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_GfpxUUipN0xUBBguLl8PkEZN', created_at=1703153152, metadata={}, object='thread')


In [10]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you compare the financial health of Microsoft and Apple over the last four years, focusing on their balance sheets and key financial ratios?"
)

In [11]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [20]:
import time
# Loop until the run completes or requires action
while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
        print("Run Steps:", run_steps)

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**function_args)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            # List the messages to get the response
            
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run
            
                   
        
            
            

        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break

Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_UJfZxLuc6Zs3mlAU1iJy47MG', assistant_id='asst_CgKwlZtBPP0IyncDukVBI0cI', cancelled_at=None, completed_at=1703153219, created_at=1703153204, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_WZL3Q5O1ucBepVxjdvbLGFqL', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_HjetGA15CvnkI899aB1Lohc7'), type='message_creation'), thread_id='thread_GfpxUUipN0xUBBguLl8PkEZN', type='message_creation', expires_at=None), RunStep(id='step_C6t0xESZIuGXS8mFGf8omPk3', assistant_id='asst_CgKwlZtBPP0IyncDukVBI0cI', cancelled_at=None, completed_at=1703153201, created_at=1703153194, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_WZL3Q5O1ucBepVxjdvbLGFqL', status='completed', step_details=ToolCallsStepDetails(tool_calls=[FunctionToolCall(id='call_59AfoFuEzoeY9WBxOzPJJ4fO', function=Functi